In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact


v, f = igl.read_triangle_mesh("gaudi.ply")

k = igl.gaussian_curvature(v, f)
plot(v, f, k)

c:\Users\Utente\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.71693…

In [7]:
v1, v2, k1, k2 = igl.principal_curvature(v, f)
h2 = 0.5 * (k1 + k2)
p = plot(v, f, h2, shading={"wireframe": False}, return_plot=True)

avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.71693…

2

In [3]:
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
# Compute Laplacian and Area Matrix
L = -igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# Compute EigenDecomposition
try:
    evals, evecs = eigsh(L, 200, M, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals, evecs = eigsh(L- 1e-8* scipy.sparse.identity(v_src.shape[0]), 200,
                         M, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
    
     

# Visualization
plot(v,f,evecs[:,1])
     


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.71693…

In [8]:
# Coordinate low pass representation
evecs_trim = evecs[:,0:40]
v1 = np.matmul(evecs_trim,np.matmul(evecs_trim.T,csr_matrix.dot(M,v)))
subplot(v1, f, s=[1, 2, 0])

v2 = np.matmul(evecs,np.matmul(evecs.T,csr_matrix.dot(M,v)))
subplot(v2, f, s=[1, 2, 1])

In [9]:

## Select a vertex from which the distances should be calculated
vs = np.array([0])
##All vertices are the targets
vt = np.arange(v.shape[0])

d = igl.exact_geodesic(v, f, vs, vt)
plot(v,f,d)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.71693…

In [10]:
g = igl.grad(v, f)
gu = g.dot(d).reshape(f.shape, order="F")
gu_mag = np.linalg.norm(gu, axis=1)

p = plot(v, f, d, shading={"wireframe":False}, return_plot=True)

max_size = igl.avg_edge_length(v, f) / np.mean(gu_mag)
bc = igl.barycenter(v, f)
bcn = bc + max_size * gu
p.add_lines(bc, bcn, shading={"line_color": "black"});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.71693…